In [7]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
#print(os.listdir("../input"))


def determineShift(hour):
    if 0<=hour<8:
        return 0
    if 8<=hour<16:
        return 1
    else:
        return 2


crime_file = '/tmp/STL_Crime_2015-2018.csv'
usecols=['DateOccur', 'Neighborhood']

print("creating datafrom from file.")
df = pd.read_csv(crime_file,usecols=usecols,parse_dates= ['DateOccur',])
print("dataframe was created from file")
#remove all neighorhoods but one for debugging
df.drop(df[df.Neighborhood > 2].index, inplace=True)

df.reset_index(drop=True,inplace=True)
#Remove any rows that have effed up coordinates
df.drop(df[df.Neighborhood == 0].index,inplace=True)

print (df.DateOccur[0])

#convert the string-typed 'DateOccur' to a datetime
#print("converting DateOccur to datetime")
#df['DateOccur'] = pd.to_datetime(df['DateOccur'])
print("adding shifts")
df['Shift'] = df['DateOccur'].map(lambda x: determineShift(x.hour))
print("searching for start/end dates")
start_date = df['DateOccur'].min().date()
end_date = df['DateOccur'].max().date()
print("startDate",start_date);
print("endDate",end_date);


df['just_date'] = df['DateOccur'].dt.date
g1 = pd.DataFrame({'count' : df.groupby( [ "Neighborhood","just_date","Shift"] ).size()}).reset_index()

 
#need to fill in the blanks, for every date/shift/neighborhood that is missing from the dataset, add a count of 0
#set the index of the dataframe to neighborhood,date, shift (multiindex)
g1.set_index(['just_date','Neighborhood','Shift'], inplace=True)
d = start_date
delta = datetime.timedelta(days=1)
nbhds = df['Neighborhood'].unique();
gap_rows_list = []
print("starting to fill in the gaps")
while d <= end_date:
  #print(d)
  for n in nbhds:
    #print(' '+str(n))
    for s in range(0,3):
      #print('  s')
      #check if the index is in the dataframe
      if g1.index.isin([(d, n, s)]).any() == False:
        #the index wasn't found in the frame.  Add a row to the gap_list with a count of 0 
        gap_rows_list.append({'Neighborhood':n,'just_date':d,'Shift':s,'count':0})
  d += delta
#at this point, gap_rows_list should contain a row for each day/neighborhood/shift with a count=0
#Create a dataframe from gap_rows_list
gap_df = pd.DataFrame( gap_rows_list)
g1.reset_index(inplace=True)
full_df=g1.append(gap_df,ignore_index=True)
#full_df should now have a row for each date,neigborhood,shift with a crime count
#this is the df that we should be working with going forward.
#delete any other dfs we have created.

del df
del g1
del gap_df


#break up the just_date into Month,Day,Year columns
full_df['Month'] = full_df['just_date'].map(lambda x: x.month)
full_df['Year'] = full_df['just_date'].map(lambda x: x.year)
full_df['Day'] = full_df['just_date'].map(lambda x: x.day)
full_df.drop(columns=['just_date'],inplace=True)

#rearrange the columns to make some order
full_df = full_df[['Year','Month','Day','Neighborhood','Shift','count']]

print(full_df)
full_df.to_csv('out.csv',index=False)


creating datafrom from file.
dataframe was created from file
2018-12-31 20:30:00
adding shifts
searching for start/end dates
startDate 2015-01-01
endDate 2018-12-31
starting to fill in the gaps
      Year  Month  Day  Neighborhood  Shift  count
0     2015      1    1             1      0      1
1     2015      1    1             1      1      3
2     2015      1    1             1      2      2
3     2015      1    2             1      0      2
4     2015      1    2             1      1      2
5     2015      1    2             1      2      2
6     2015      1    3             1      1      6
7     2015      1    4             1      0      1
8     2015      1    4             1      2      1
9     2015      1    5             1      0      1
10    2015      1    5             1      1      4
11    2015      1    5             1      2      1
12    2015      1    6             1      0      3
13    2015      1    6             1      1      1
14    2015      1    7             1     

Load up the full data

In [26]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

dataset = np.loadtxt("out.csv", delimiter=",",skiprows=1,dtype=int )

np.random.shuffle(dataset)

print (len(dataset))
trainSize = int(round(len(dataset)*.3))
valSize = int(round(len(dataset)*.2))
testSize = int(round(len(dataset)*.5))


XTRAIN = dataset[:trainSize,0:5]
YTRAIN = dataset[:trainSize,5]
XVALIDATION = dataset[trainSize:valSize,0:5]
YVALIDATION = dataset[trainSize:valSize,5]
XTEST = dataset[:testSize,0:5]
YTEST = dataset[:testSize,5]

print(XTRAIN)

model = Sequential()
model.add(Dense(12, input_dim=8, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(1, activation='linear'))
print(model.summary())

#print(XTRAIN.shape)

ModuleNotFoundError: No module named 'keras'

Split up data into training data, validation data, and testing data